In [139]:
import pandas as pd
%load_ext autoreload
%autoreload 2
import torch
from transformers import GPTNeoForSequenceClassification, GPTNeoForCausalLM, GPT2Tokenizer
from model import GPTNeoForSequenceClassificationBinary
import lightning.pytorch as pl
from model import GPTSequenceClassifiationModule

test_name = 'data/wordnet_all_false_july4_test.csv' #'data/ccs_test.csv'
actual_tree = 'data/bansal_wordnet_true_pairs.csv' #'data/acm_ccs_clean.csv'
model_path = '/root/autodl-tmp/runs/lora_8.ckpt'
save_path = 'results/wordnet/lora.csv'

model = GPTSequenceClassifiationModule.load_from_checkpoint(model_path)
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Some weights of GPTNeoForSequenceClassification were not initialized from the model checkpoint at EleutherAI/gpt-neo-1.3B and are newly initialized: ['score.weight', 'transformer.h.13.attn.attention.bias', 'transformer.h.7.attn.attention.bias', 'transformer.h.19.attn.attention.bias', 'transformer.h.3.attn.attention.bias', 'transformer.h.1.attn.attention.bias', 'transformer.h.21.attn.attention.bias', 'transformer.h.9.attn.attention.bias', 'transformer.h.5.attn.attention.bias', 'transformer.h.11.attn.attention.bias', 'transformer.h.17.attn.attention.bias', 'transformer.h.15.attn.attention.bias', 'transformer.h.23.attn.attention.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1576962 || all params: 1317152770 || trainable%: 0.11972506423837229
PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): GPTNeoForSequenceClassification(
      (transformer): GPTNeoModel(
        (wte): Embedding(50257, 2048)
        (wpe): Embedding(2048, 2048)
        (drop): Dropout(p=0.0, inplace=False)
        (h): ModuleList(
          (0-23): 24 x GPTNeoBlock(
            (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (attn): GPTNeoAttention(
              (attention): GPTNeoSelfAttention(
                (attn_dropout): Dropout(p=0.0, inplace=False)
                (resid_dropout): Dropout(p=0.0, inplace=False)
                (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
                (v_proj): Linear(
                  in_features=2048, out_features=2048, bias=False
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
              

In [140]:

model=model.eval()
model=model.cuda()

In [141]:
# test_name='/root/Taxonomy-GPT/data/wordnet_all_false_july4_test.csv'
# actual_tree='/root/Taxonomy-GPT/data/bansal_wordnet_true_pairs.csv'
# test_name='/root/Taxonomy-GPT/data/ccs_test.csv'
# actual_tree='/root/Taxonomy-GPT/data/acm_ccs_clean.csv'

In [142]:
from dataset import get_taxonomy_dataset, get_taxonomy_dataset_binary
dataset = get_taxonomy_dataset_binary(test_name, entire_dataset=True,remove_columns=True)

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-78e6d1eb18fc7984/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-78e6d1eb18fc7984/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-efeade2dc67e1dd3.arrow


In [143]:
dataset

{'test': Dataset({
     features: ['flag', 'input_ids', 'labels'],
     num_rows: 53968
 })}

In [144]:
def format_dataset(example,tokenizer):
    prompt_template = "I am doing the taxonomy research. I think {child} is a subtopic of {parent}"
    prompt = prompt_template.format(child=example['child'], parent=example['parent'])
    example['input_ids'] = tokenizer.encode(prompt)
    return example

def test_dataloader(str_path,batch_size):
    dataset= get_taxonomy_dataset_binary(str_path,entire_dataset=True)
    dataset = dataset['test']
    dataset = dataset.map(lambda example: format_dataset(example,tokenizer))
    dataset = dataset.remove_columns(['child', 'parent', 'group'])

    return torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        collate_fn=DataCollatorWithPadding(tokenizer),
        num_workers=4,
    )

In [145]:
from torch.utils.data import DataLoader
from transformers.data.data_collator import DataCollatorWithPadding
from tqdm.notebook import tqdm
import torch 

results = []
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorWithPadding(tokenizer)

testdataloader = test_dataloader(test_name, 64)
with torch.no_grad():
    for step, batch in enumerate(tqdm(testdataloader)):
        batch = batch.to('cuda')
        output = model(batch['input_ids'], batch['attention_mask'])
        predicted_labels = torch.sigmoid(output)
        results.extend(predicted_labels.cpu().detach().numpy().tolist())

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-78e6d1eb18fc7984/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-78e6d1eb18fc7984/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-efeade2dc67e1dd3.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-78e6d1eb18fc7984/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-67b183b76ee90bd1.arrow


  0%|          | 0/844 [00:00<?, ?it/s]

In [146]:
import pandas as pd
df = pd.read_csv(test_name)
df['predict'] = results
df['pred_flag']=[True if i>0.5 else False for i in df['predict']]

In [147]:
df.to_csv(save_path, index=False)

In [148]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
y_test=df['flag']
y_pred=df['pred_flag']
print(f1_score(y_test, y_pred, average="macro"))
print(precision_score(y_test, y_pred, average="macro"))
print(recall_score(y_test, y_pred, average="macro"))
print(accuracy_score(y_test, y_pred))

0.639635335965705
0.6056394770396241
0.7610640290182216
0.9004780610732286


In [149]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.98      0.91      0.95     51767
        True       0.23      0.61      0.33      2201

    accuracy                           0.90     53968
   macro avg       0.61      0.76      0.64     53968
weighted avg       0.95      0.90      0.92     53968



maximum spanning tree

In [150]:
df

,parent,child,group,flag,predict,pred_flag
0,explosion,blowback,647,True,0.974382,True
1,explosion,blast,647,True,0.995553,True
2,explosion,bomb_blast,647,False,0.988489,True
3,explosion,airburst,647,True,0.986376,True
4,explosion,big_bang,647,True,0.991391,True
...,...,...,...,...,...,...
53963,linolenic_acid,oleic_acid,760,False,0.129126,False
53964,linolenic_acid,linoleic_acid,760,False,0.393263,False
53965,linolenic_acid,docosahexaenoic_acid,760,False,0.393662,False
53966,linolenic_acid,eicosapentaenoic_acid,760,False,0.300765,False


In [151]:
df_t=df

def convert_to_ancestor_graph(G):
    '''Converts a (parent) tree to a graph with edges for all ancestor relations in the tree.'''
    G_anc = nx.DiGraph()
    for node in G.nodes():
        for anc in nx.ancestors(G, node):
            G_anc.add_edge(anc, node)
    return G_anc

In [152]:
import networkx as nx
import numpy as np

forest=[]
for g in tqdm(list(set(df_t.group))):
    df_tree=df_t[df_t.group==g]
    graph = nx.DiGraph()
    parents=df_tree['parent'].tolist()
    children=df_tree['child'].tolist()
    probabilities=df_tree['predict'].tolist()
    nodes = set(parents + children)
    for node in nodes:
        graph.add_node(node)
    for i in range(len(parents)):
        graph.add_edge(parents[i], children[i], weight=np.log(probabilities[i]))
    # print(nx.is_connected(graph))
    T=nx.maximum_spanning_arborescence(graph)
    T = convert_to_ancestor_graph(T)
    df=nx.to_pandas_edgelist(T)
    df['group']=g
    forest.append(df)

  0%|          | 0/114 [00:00<?, ?it/s]

In [153]:
res_v2=pd.concat(forest, ignore_index=True)


In [154]:
res_v2

,source,target,group
0,explosion,inflation,647
1,explosion,big_bang,647
2,explosion,nuclear_explosion,647
3,explosion,bomb_blast,647
4,explosion,backblast,647
...,...,...,...
3570,unsaturated_fatty_acid,docosahexaenoic_acid,760
3571,unsaturated_fatty_acid,elaidic_acid,760
3572,unsaturated_fatty_acid,monounsaturated_fatty_acid,760
3573,unsaturated_fatty_acid,omega-3_fatty_acid,760


In [155]:
res_v2.columns=['parent','child','group']
res_v2['compare']=res_v2['parent']+res_v2['child']+res_v2['group'].astype(str)
res_v2['group'].value_counts()

group
697    116
712    108
666    103
667     89
740     71
      ... 
738     11
682     11
668     11
746     10
647     10
Name: count, Length: 114, dtype: int64

In [156]:
df_actual=pd.read_csv(actual_tree)
df_actual=df_actual[df_actual['type'] == 'test']
df_actual['compare']=df_actual['parent']+df_actual['child']+df_actual['group'].astype(str)

In [157]:
forest=[]
for g in tqdm(list(set(df_actual.group))):
    df_tree=df_actual[df_actual.group==g]
    graph = nx.DiGraph()
    parents=df_tree['parent'].tolist()
    children=df_tree['child'].tolist()
    nodes = set(parents + children)
    for node in nodes:
        graph.add_node(node)
    for i in range(len(parents)):
        graph.add_edge(parents[i], children[i])
    # print(nx.is_connected(graph))
    T = convert_to_ancestor_graph(graph)
    df=nx.to_pandas_edgelist(T)
    df['group']=g
    forest.append(df)

df_actual =pd.concat(forest, ignore_index=True)
df_actual.columns=['parent','child','group']
df_actual['compare']=df_actual['parent']+df_actual['child']+df_actual['group'].astype(str)
df_actual['group'].value_counts()

  0%|          | 0/114 [00:00<?, ?it/s]

group
677    87
666    87
758    78
741    76
691    76
       ..
746    15
682    14
647    14
737    13
661    13
Name: count, Length: 114, dtype: int64

In [158]:
df_actual[df_actual.group=='10010147_3']

,parent,child,group,compare


In [159]:
res_v2[res_v2.group=='10010147_3']

,parent,child,group,compare


In [160]:
df_actual.merge(res_v2, on='compare',how='left').to_csv('/root/Taxonomy-GPT/data/group_wise_july_05.csv')

In [161]:
len(df_actual.merge(res_v2, on='compare'))/len(df_actual)

0.49491441329694863

In [162]:
len(df_actual.merge(res_v2, on='compare'))/len(res_v2)

0.558041958041958

In [163]:
recall = []
precision = []
f1 = []
for group in tqdm(list(set(df_actual.group))):
    group_actual = df_actual[df_actual.group == group]
    group_pred = res_v2[res_v2.group == group]
    recall.append(len(group_actual.merge(group_pred, on='compare')) / len(group_actual))
    precision.append(len(group_actual.merge(group_pred, on='compare')) / len(group_pred))
    if precision[-1] + recall[-1] == 0:
        f1.append(0)
    else:
        f1.append(2 * (precision[-1] * recall[-1]) / (precision[-1] + recall[-1]))

  0%|          | 0/114 [00:00<?, ?it/s]

In [164]:
import numpy as np
np.mean(recall)

0.5121387919282124

In [165]:
np.mean(precision)

0.6256533225972404

In [166]:
np.mean(f1)

0.550320877862728